<a href="https://colab.research.google.com/github/machiwao/CCTHESS1-CCTHESS2-Dev-and-Docs/blob/coli/'14_'23_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install earthengine-api geemap --quiet


# Install the Google Earth Engine-API

In [1]:
import ee
import geemap
import pandas as pd


# Authenticate and Initialize

In [2]:
ee.Authenticate()
ee.Initialize(project='data-scrapping-470316')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


# Define Location and Buffer

In [16]:
point = ee.Geometry.Point([121.630454, 18.360059])  # longitude, latitude for Aparri
buffer_radius_m = 5000  # Buffer 5 km
region = point.buffer(buffer_radius_m)

# Define Time Range

In [17]:
start_date = '2014-01-01'
end_date = '2023-12-31'

# Fetch dataset and bands

In [52]:
dataset = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filterDate(start_date, end_date)
bands = dataset.first().bandNames().getInfo()
print("Available Bands:", bands)

Available Bands: ['dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature', 'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4', 'lake_bottom_temperature', 'lake_ice_depth', 'lake_ice_temperature', 'lake_mix_layer_depth', 'lake_mix_layer_temperature', 'lake_shape_factor', 'lake_total_layer_temperature', 'snow_albedo', 'snow_cover', 'snow_density', 'snow_depth', 'snow_depth_water_equivalent', 'snowfall_sum', 'snowmelt_sum', 'temperature_of_snow_layer', 'skin_reservoir_content', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'forecast_albedo', 'surface_latent_heat_flux_sum', 'surface_net_solar_radiation_sum', 'surface_net_thermal_radiation_sum', 'surface_sensible_heat_flux_sum', 'surface_solar_radiation_downwards_sum', 'surface_thermal_radiation_downwards_sum', 'evaporation_from_bare_soil_sum', 'evaporation_from_open_water_surfaces_excludi

# Conversion of Kelvin to Celcius

In [57]:
def convert_temperature_bands_drop_kelvin(img):
    # Convert temperature_2m_min and temperature_2m_max to Celsius with new names
    temp_min_c = img.select('temperature_2m_min').subtract(273.15).rename('temperature_2m_min')
    temp_max_c = img.select('temperature_2m_max').subtract(273.15).rename('temperature_2m_max')

    # Select all bands except the original Kelvin bands
    other_bands = img.select(img.bandNames().removeAll(['temperature_2m_min', 'temperature_2m_max']))

    # Combine other bands with converted bands (Celsius)
    return other_bands.addBands([temp_min_c, temp_max_c])

# Map over images extract band values at location

In [62]:
def image_to_feature(img):
    stats = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=1000,
        maxPixels=1e9
    )
    # Create formatted date string
    formatted_date = ee.Date(img.get('system:time_start')).format('yyyy-MM-dd')

    # Remove system:index by creating new dict without it
    stats = ee.Dictionary(stats).set('date', formatted_date)

    return ee.Feature(None, stats)


features = converted_dataset.map(image_to_feature)
fc = ee.FeatureCollection(features)

# Export to Drive as CSV

In [64]:
#out_csv = '/content/era5_aparri_time_series.csv'
out_csv = '/content/era5_aparri_daily_all_bands.csv'
geemap.ee_export_vector(fc, out_csv, verbose=True)
print(f"CSV saved to {out_csv}")

Generating URL ...
Please wait ...
Data downloaded to /content/era5_aparri_daily_all_bands.csv
CSV saved to /content/era5_aparri_daily_all_bands.csv
